In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rc('font', family='NanumGothic')
import plotly.express as px

import datetime
import plotly.express as px

import plotly
from plotly.graph_objs import Scatter, Line, Layout
 
import importlib
import json
 
from plotly.subplots import make_subplots
import plotly.graph_objects as go


In [2]:
fpopl=pd.read_csv('fpopl.csv',usecols=['base_ymd','adstrd_code','popltn_cascnt'])
fpopl.head()

,base_ymd,adstrd_code,popltn_cascnt
0,20200101,11650560,15
1,20200101,11590620,6
2,20200101,11560710,4
3,20200101,11470680,12
4,20200101,11350665,6


날짜별 유동인구 수

fpopl_b : 코로나 확산 이전, fpopl_a : 코로나 확신이후

In [3]:
is_day=fpopl['base_ymd']<20200311
fpopl_b=fpopl[is_day]
fpopl_b.tail()

,base_ymd,adstrd_code,popltn_cascnt
19796789,20200310,11320670,48
19796790,20200310,11380520,22
19796791,20200310,11470610,8
19796792,20200310,11500535,52
19796793,20200310,11560700,22


In [4]:
fpopl_b['adstrd_code']=fpopl_b['adstrd_code'].astype(str)
fpopl_b['adstrd_code'] = fpopl_b['adstrd_code'].apply(lambda x:x[:5])

In [5]:
fpopl_b=fpopl_b.groupby(by=['adstrd_code'],as_index=False).popltn_cascnt.mean()

In [6]:
fpopl_b.head()

,adstrd_code,popltn_cascnt
0,11110,147.023546
1,11140,184.023932
2,11170,218.085938
3,11200,201.388781
4,11215,199.497453


In [7]:
fpopl_a=fpopl.drop(fpopl[is_day].index)

In [8]:
fpopl_a['adstrd_code']=fpopl_a['adstrd_code'].astype(str)
fpopl_a['adstrd_code']=fpopl_a['adstrd_code'].apply(lambda x:x[:5])

In [9]:
fpopl_a=fpopl_a.groupby(by=['adstrd_code'],as_index=False).popltn_cascnt.mean()
fpopl_a.head()

,adstrd_code,popltn_cascnt
0,11110,145.563854
1,11140,176.336182
2,11170,196.987985
3,11200,218.802406
4,11215,201.441064


In [10]:
add_2=[]
for i in range(len(fpopl_a)):
    add_2.append(fpopl_a['popltn_cascnt'][i])

fpopl_diff=fpopl_b
fpopl_diff['covid_after']=add_2

fpopl_diff.columns=['adstrd_code','covid_before','covid_after']
fpopl_diff.head()

,adstrd_code,covid_before,covid_after
0,11110,147.023546,145.563854
1,11140,184.023932,176.336182
2,11170,218.085938,196.987985
3,11200,201.388781,218.802406
4,11215,199.497453,201.441064


In [11]:
fpopl_diff['diff']=fpopl_diff['covid_before']-fpopl_diff['covid_after']
fpopl_diff=fpopl_diff.sort_values(by=['diff'],ascending=False)
fpopl_diff=fpopl_diff.reset_index()
fpopl_diff=fpopl_diff.drop(['index'],axis=1)
fpopl_diff.head()

,adstrd_code,covid_before,covid_after,diff
0,11170,218.085938,196.987985,21.097953
1,11650,317.802200,302.648947,15.153254
2,11530,189.388577,175.724644,13.663933
3,11140,184.023932,176.336182,7.687749
4,11560,236.304922,233.458827,2.846095


In [12]:
adstrd=pd.read_csv('adstrd_master.csv')
adstrd['adstrd_code']=adstrd['adstrd_code'].astype(str)
adstrd['adstrd_code']=adstrd['adstrd_code'].apply(lambda x:x[:5])
adstrd['total_adstrd']=adstrd['brtc_nm']+' '+adstrd['signgu_nm']

adstrd_s=adstrd[adstrd['brtc_nm']=='서울특별시']
adstrd_s=adstrd_s[['adstrd_code','total_adstrd']]

In [13]:
adstrd_s=adstrd_s.drop_duplicates(['adstrd_code'],keep='first')
adstrd_s=adstrd_s.reset_index()
adstrd_s=adstrd_s.drop(['index'],axis=1)
adstrd_s.head(10)

,adstrd_code,total_adstrd
0,11110,서울특별시 종로구
1,11140,서울특별시 중구
2,11170,서울특별시 용산구
3,11200,서울특별시 성동구
4,11215,서울특별시 광진구
5,11230,서울특별시 동대문구
6,11260,서울특별시 중랑구
7,11290,서울특별시 성북구
8,11305,서울특별시 강북구
9,11320,서울특별시 도봉구


코로나 확산 이전/이후 유동인구변화 값

In [14]:
adstrd_nm=dict()
for i in range(len(adstrd_s)):
    adstrd_nm[adstrd_s['adstrd_code'][i]]=adstrd_s['total_adstrd'][i]
  
ad_name=[]
for i in range(len(fpopl_diff)):
    ad_name.append(adstrd_nm[fpopl_diff['adstrd_code'][i]])
fpopl_diff['adstrd_name']=ad_name

In [15]:
fpopl_diff=fpopl_diff.sort_values(by=['adstrd_name'],ascending=False)
fpopl_diff=fpopl_diff.reset_index()
fpopl_diff=fpopl_diff.drop(['index'],axis=1)

In [16]:
region=pd.read_csv('region.csv',usecols=['province','city','latitude','longitude'])
region_seoul=region[region['province']=='서울']
region_seoul['province']='서울특별시'
region_seoul['total_ads']=region_seoul['province']+' '+region_seoul['city']
region_s=region_seoul[['total_ads','latitude','longitude']]
region_s=region_s[1:]
region_s=region_s.reset_index()
region_s=region_s.drop(['index'],axis=1)
region_s=region_s.sort_values(by=['total_ads'],ascending=False)
region_s=region_s.reset_index()
region_s=region_s.drop(['index'],axis=1)

In [17]:
ad_lat=[]
ad_lon=[]
for i in range(len(region_s)):
    ad_lat.append(region_s['latitude'][i])
for i in range(len(region_s)):
    ad_lon.append(region_s['longitude'][i])

fpopl_diff['latitude']=ad_lat
fpopl_diff['longitude']=ad_lon

In [18]:
fpopl_diff=fpopl_diff.sort_values(by=['diff'],ascending=False)
fpopl_diff

,adstrd_code,covid_before,covid_after,diff,adstrd_name,latitude,longitude
4,11170,218.085938,196.987985,21.097953,서울특별시 용산구,37.532768,126.990021
10,11650,317.802200,302.648947,15.153254,서울특별시 서초구,37.483804,127.032693
18,11530,189.388577,175.724644,13.663933,서울특별시 구로구,37.495632,126.887650
1,11140,184.023932,176.336182,7.687749,서울특별시 중구,37.563988,126.997530
5,11560,236.304922,233.458827,2.846095,서울특별시 영등포구,37.526505,126.896190
7,11710,199.632135,197.795685,1.836451,서울특별시 송파구,37.514620,127.106141
11,11410,198.367067,196.546442,1.820625,서울특별시 서대문구,37.579428,126.936771
13,11590,179.320593,177.597494,1.723099,서울특별시 동작구,37.574552,127.039721
2,11110,147.023546,145.563854,1.459692,서울특별시 종로구,37.572999,126.979189
20,11620,134.682392,133.794927,0.887465,서울특별시 관악구,37.478290,126.951502
